In [1]:
import urllib
import zipfile
import geopandas as gpd
import pandas as pd

/Users/angelserrano/opt/miniconda3/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


url_download = r'https://www.inegi.org.mx/contenidos/programas/ccpv/2020/microdatos/ageb_manzana/RESAGEBURB_16_2020_csv.zip'
input_path = 'inputs/'
urllib.request.urlretrieve(url_download, 'RESAGEBURB_16_2020_csv.zip')
unzip = zipfile.ZipFile('RESAGEBURB_16_2020_csv.zip')
unzip.extractall(input_path)

url_download = r'https://www.inegi.org.mx/contenidos/productos/prod_serv/contenidos/espanol/bvinegi/productos/geografia/marcogeo/889463849568/16_michoacandeocampo.zip'
input_path = 'inputs/'
urllib.request.urlretrieve(url_download, '16_michoacandeocampo.zip')
unzip = zipfile.ZipFile('16_michoacandeocampo.zip')
unzip.extractall(input_path)

In [2]:
michoacan_full = pd.read_csv('inputs/RESAGEBURB_16CSV20.csv')

In [3]:
michoacan_shp = gpd.read_file('inputs/conjunto_de_datos/16m.shp')

In [4]:
morelia = michoacan_full.loc[(michoacan_full["MUN"] ==53) & (michoacan_full["MZA"] !=0) ]

In [5]:
morelia["ENTIDAD"] = morelia["ENTIDAD"].astype(str)

/Users/angelserrano/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [6]:
morelia['MUN'] = morelia['MUN'].apply(lambda x: '{0:0>3}'.format(x))
morelia['LOC'] = morelia['LOC'].apply(lambda x: '{0:0>4}'.format(x))
morelia['MZA'] = morelia['MZA'].apply(lambda x: '{0:0>3}'.format(x))
morelia['CVEGEO']= morelia['ENTIDAD'] + morelia['MUN'] +morelia['LOC'] +morelia['AGEB'] +morelia['MZA']


/Users/angelserrano/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/angelserrano/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/angelserrano/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

In [7]:
morelia_shp = michoacan_shp.loc[(michoacan_shp["CVE_MUN"] =='053') ]

In [8]:
shape_full = pd.merge(morelia_shp, morelia[['MUN','LOC','MZA','CVEGEO','POBTOT','P_18YMAS','TVIVHAB']], on = 'CVEGEO')

In [9]:
morelia_urbano = shape_full.loc[(shape_full["AMBITO"] =='Urbana') ]

In [10]:
morelia_urbano = morelia_urbano.replace({'*': '0'})

In [11]:
morelia_urbano_habitado = morelia_urbano.loc[(morelia_urbano["TVIVHAB"] !=0) ]

In [12]:
colonias_michoacan = gpd.read_file("inputs/16-MICHOACAN/COLONIA.shp" )

In [13]:
colonias_morelia = colonias_michoacan.loc[(colonias_michoacan["MUNICIPIO"] ==54) ]


In [14]:
colonias_morelia = colonias_morelia.to_crs('EPSG:4326')


In [15]:
morelia_urbano_habitado = morelia_urbano_habitado.to_crs('EPSG:4326')

In [16]:
morelia_urbano_habitado['centroid'] = morelia_urbano_habitado.centroid
join = gpd.sjoin(colonias_morelia, morelia_urbano_habitado.set_geometry('centroid'), how="right", predicate="contains")


/Users/angelserrano/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """Entry point for launching an IPython kernel.


In [17]:
join.P_18YMAS = join.P_18YMAS.fillna(0)

In [18]:
join.TVIVHAB = join.TVIVHAB.fillna(0)

In [19]:
join.P_18YMAS = join.P_18YMAS.astype('int64')

In [20]:
join.TVIVHAB = join.TVIVHAB.astype('int64')

In [21]:
habitantes_colonia = join.groupby(['NOMBRE'])[['POBTOT', "P_18YMAS", "TVIVHAB"]].agg('sum').reset_index()

In [22]:
pob_col_mor = pd.merge(colonias_morelia, habitantes_colonia,on='NOMBRE')

In [23]:

cuadrantes_morelia = gpd.read_file("inputs/redudasproyectomorelia (1)/sectores_processed.shp" )




In [24]:
cuadrantes_morelia = cuadrantes_morelia.to_crs('EPSG:4326')

In [25]:
pob_col_mor['centroid'] = pob_col_mor.centroid

/Users/angelserrano/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """Entry point for launching an IPython kernel.


In [26]:

join_cuadrantes = gpd.sjoin(cuadrantes_morelia, pob_col_mor.set_geometry('centroid'), how="left", predicate="contains")

In [28]:

join_cuadrantes.groupby('Sector')['POBTOT',"P_18YMAS",'TVIVHAB'].sum()

/Users/angelserrano/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,POBTOT,P_18YMAS,TVIVHAB
Sector,,,
Centro,28630,23927,8856
Independencia,88232,69133,27245
Independencia Sur,132645,90295,40879
Nueva España,125881,93519,38140
República,180027,129026,49794
República Poniente,144577,100454,39991
Revolución,189710,131691,51853
